In [160]:
# Importing the required packages and libraries
# we will need numpy and pandas later
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# The data from the challenge was already pre split into test and train
# The difference being that test has the price removed from the data
cars_train_df = pd.read_csv('trainCopyCleaned.csv')
cars_test_df = pd.read_csv('testCopyCleaned.csv')

cars_train_df_original = pd.read_csv('train.csv')
cars_test_df_original = pd.read_csv('test.csv')


# Checking the first few rows to confirm it loaded correctly
print("Training data set")
print(cars_train_df.head())

print("Test dataset")
print(cars_test_df.head())

Training data set
   id          brand              model  model_year  milage      fuel_type  \
0   0           MINI      Cooper S Base        2007  213000       Gasoline   
1   1        Lincoln              LS V8        2002  143250       Gasoline   
2   2      Chevrolet  Silverado 2500 LT        2002  136731  E85 Flex Fuel   
3   3        Genesis   G90 5.0 Ultimate        2017   19500       Gasoline   
4   4  Mercedes-Benz        Metris Base        2021    7388       Gasoline   

                                              engine  \
0       172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel   
1       252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel   
2  320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...   
3       420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel   
4       208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel   

                     transmission ext_col int_col  \
0                             A/T  Yellow    Gray   
1                             A/T  Silver   Beige   
2                

The first thing we notice is that one the id column is repetitive and not necessary, second there is a lot of columns or features with missing values or just NAN we must account for this in our clean up. For example there are entries with no accidents reported, yet the clean title column is blank or vice versa. This sort of leads us to belive the clean title feature will not be that useful or have a low impact since it can at times contradict the accident feature. In real world scenario this is more complex as you can have an accident but still a clean title, but is just something for us to consider with our model. We will think of possibly cleaning or combining this feaure in the future.

Just to give ourselves a better understanding we will look at the average price of the whole training set, as well as by brand. This step is purely just for ourselves to give us a rough ballpark of the values.

In [163]:
# average price of all cars
average_price_all_cars = cars_train_df['price'].mean()
print(f"Average price of all cars: ${average_price_all_cars:.2f}")

# Lowest and highest price
lowest_price = cars_train_df['price'].min()
highest_price = cars_train_df['price'].max()

print(f"\nLowest price: ${lowest_price:.2f}")
print(f"Highest price: ${highest_price:.2f}")

# average price by brand
average_price_by_brand = cars_train_df.groupby('brand')['price'].mean()

print("\nAverage price by brand:")
for brand, avg_price in average_price_by_brand.items():
    print(f"{brand}: ${avg_price:.2f}")

Average price of all cars: $42605.36

Lowest price: $2000.00
Highest price: $1599000.00

Average price by brand:
Acura: $28628.50
Alfa: $51993.10
Aston: $100374.22
Audi: $40737.26
BMW: $43323.68
Bentley: $112486.64
Bugatti: $169600.00
Buick: $29617.84
Cadillac: $42350.53
Chevrolet: $40772.54
Chrysler: $22978.95
Dodge: $32344.97
FIAT: $31045.89
Ferrari: $116751.01
Ford: $38878.67
GMC: $38029.15
Genesis: $53556.33
Honda: $21405.12
Hummer: $15355.61
Hyundai: $24503.95
INFINITI: $32479.86
Jaguar: $48147.54
Jeep: $34116.92
Karma: $37384.18
Kia: $38072.47
Lamborghini: $156958.41
Land: $51681.31
Lexus: $33046.14
Lincoln: $30712.60
Lotus: $26436.48
Lucid: $63310.36
MINI: $17162.02
Maserati: $61459.70
Maybach: $53566.33
Mazda: $24624.34
McLaren: $123261.24
Mercedes-Benz: $49938.90
Mercury: $26081.79
Mitsubishi: $15398.74
Nissan: $29246.56
Plymouth: $28095.00
Polestar: $41754.86
Pontiac: $17069.07
Porsche: $68416.33
RAM: $47056.55
Rivian: $81089.15
Rolls-Royce: $132246.67
Saab: $28594.00
Saturn:

As mentioned before we are now going to look at how many features have NAN or no input, again this is just additional to help us understand our data better.

In [166]:
# Total number of missing values in the training set
missing_train = cars_train_df.isna().sum().sum()

# Total number of missing values in the test set
missing_test = cars_test_df.isna().sum().sum()


#showing the missing data by column
print("\nMissing values by column (training dataset cleaned):")
print(cars_train_df.isnull().sum())

print("\nMissing values by column (test dataset cleaned):")
print(cars_test_df.isnull().sum())


# Total number of missing values in the training set original
missing_train_og = cars_train_df_original.isna().sum().sum()

# Total number of missing values in the test set
missing_test_og = cars_test_df_original.isna().sum().sum()


#showing the missing data by column
print("\nMissing values by column (training dataset original):")
print(cars_train_df_original.isnull().sum())

print("\nMissing values by column (test dataset original):")
print(cars_test_df_original.isnull().sum())


Missing values by column (training dataset cleaned):
id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64

Missing values by column (test dataset cleaned):
id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
dtype: int64

Missing values by column (training dataset original):
id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

Missing values by column (test dataset original):
id             

In [168]:
# looking at the other feature now such as transmission etc
# Count the occurrences of each transmission type
# as we can see a lot of variation here as well
transmission_counts = cars_train_df['transmission'].value_counts()

# Print the counts
print(transmission_counts)


transmission
A/T                                                   49892
8-Speed A/T                                           20638
Transmission w/Dual Shift Mode                        19243
6-Speed A/T                                           18040
6-Speed M/T                                           11989
7-Speed A/T                                           11120
Automatic                                             10687
8-Speed Automatic                                      8416
10-Speed A/T                                           8042
9-Speed A/T                                            3865
5-Speed A/T                                            3212
10-Speed Automatic                                     3156
6-Speed Automatic                                      2797
4-Speed A/T                                            2544
5-Speed M/T                                            2408
9-Speed Automatic                                      2323
CVT Transmission           

In [170]:
# looking at accident count
accident_counts = cars_train_df['accident'].value_counts()
print(accident_counts)

accident
None reported                             146878
At least 1 accident or damage reported     41555
Name: count, dtype: int64


In [172]:
# seeing all the different fuel types now
fuel_type_counts = cars_train_df['fuel_type'].value_counts()

print("Fuel Type Counts:")
print(fuel_type_counts)

Fuel Type Counts:
fuel_type
Gasoline          166696
Hybrid              6842
E85 Flex Fuel       5404
Electric            5012
Diesel              3951
Plug-In Hybrid       521
Hydrogen               7
Name: count, dtype: int64


As we mentioned before the most discrepencies seem to be in accident and title type, fuel type also has some missing entries but we do not see it being as large of an issue. The most important features we guess will be brand/model, mileage, and age. We also noticed some significant outliers with prices in the millions of cars that seemed to be collectibles, luxury, or charity auction events like a ford mustang and f150 pickup in the millions. While these might be true they are not good representitives for the average used car market. In addition we have some cars with extremely high or low miles which might skew our results. We must also consider how to use the brand and model because there are so many unique values, as well as engine and transmission configuration as they are not simply automatic or manual.

In [204]:
columns_to_clean = ['milage', 'price']

for column in columns_to_clean:
    top = cars_train_df[column].quantile(0.25)
    bottom = cars_train_df[column].quantile(0.75)

    median = bottom - top

    lower_bound = top - 1.5 * median
    upper_bound = bottom + 1.5 * median

    # removing those outside said bounds
    cars_train_df = cars_train_df[(cars_train_df[column] >= lower_bound) & (cars_train_df[column] <= upper_bound)]

# reseting the indexes
cars_train_df.reset_index(drop=True, inplace=True)

# checking
print("Cleaned DataFrame:")
print(cars_train_df.head())

Cleaned DataFrame:
   id          brand              model  model_year  milage      fuel_type  \
0   1        Lincoln              LS V8        2002  143250       Gasoline   
1   2      Chevrolet  Silverado 2500 LT        2002  136731  E85 Flex Fuel   
2   3        Genesis   G90 5.0 Ultimate        2017   19500       Gasoline   
3   4  Mercedes-Benz        Metris Base        2021    7388       Gasoline   
4   5           Audi      A6 2.0T Sport        2018   40950       Gasoline   

                                              engine  \
0       252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel   
1  320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...   
2       420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel   
3       208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel   
4       252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel   

                     transmission ext_col int_col  \
0                             A/T  Silver   Beige   
1                             A/T    Blue    Gray   
2  Transmission 

Now we are using our cleaned data, previous versions of our code can be found on github

In [207]:
# Creating the Feature Matrix for iris dataset:

# create a python list of feature names that would like to pick from the dataset:
# Start by selecting the relevant columns for prediction
feature_cols = [
    'brand',       # Categorical
    'model',       # Categorical
    'model_year',  # Numeric
    'milage',      # Numeric
    'fuel_type',   # Categorical
    'engine',      # categorical
    'transmission', # Categorical
    'ext_col',     # Categorical (possibly useful for aesthetics affecting price)
    'int_col',     # Categorical
    'accident',    # Categorical
    'clean_title'  # Binary/Categorical
]

# use the above list to select the features from the original DataFrame
X = cars_train_df[feature_cols].copy()  
X_test = cars_test_df[feature_cols].copy()
# print the first 5 rows

X.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,No
1,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes
2,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes
3,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes
4,Audi,A6 2.0T Sport,2018,40950,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,White,–,None reported,Yes


In [209]:
X_test.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,Yes
3,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,Yes
4,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [211]:
# checking the size of Feature Matix X:

print(X.shape)

(175900, 11)


In [213]:
# select a Series of labels (the last column) from the DataFrame
y = cars_train_df['price']

# checking the label vector by printing every 10 values
y[::10]

0          4999
10        51500
20         9900
30        87000
40        49999
          ...  
175850    49500
175860    15500
175870    86999
175880    12900
175890    47500
Name: price, Length: 17590, dtype: int64

In [215]:
# we need to convert these complex names and brands into
# simpler values our model can understand
# not as easy as one hot encoding because we have so much variety between
# brand name, model, engine type, etc
# originally we thought to just not include them but that seemed too crude
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

# just changing the cat columns
cat_cols = [
    'brand',       # Categorical
    'model',       # Categorical
    'fuel_type',   # Categorical
    'engine',      # Categorical
    'transmission', # Categorical
    'ext_col',     # Categorical
    'int_col',     # Categorical
    'accident',    # Categorical
    'clean_title'  # Binary/Categorical
]

# Convert categorical columns to string type some sort of error occured chaning the types in test
X[cat_cols] = X[cat_cols].astype(str)
X_test[cat_cols] = X_test[cat_cols].astype(str)



# had to add the handle unknown value because some brands and models in test were not seen in train
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# Apply the OrdinalEncoder to the categorical columns
X[cat_cols] = ordinal_encoder.fit_transform(X[cat_cols].astype(str))
X_test[cat_cols] = ordinal_encoder.transform(X_test[cat_cols].astype(str))

# Check the result
print('X train')
print(X.head())

print('X test')
print(X_test.head())

X train
   brand   model  model_year  milage  fuel_type  engine  transmission  \
0   28.0   930.0        2002  143250        3.0   366.0          38.0   
1    9.0  1575.0        2002  136731        1.0   640.0          38.0   
2   16.0   758.0        2017   19500        3.0   863.0          49.0   
3   36.0  1077.0        2021    7388        3.0   259.0          23.0   
4    3.0   182.0        2018   40950        3.0   364.0          38.0   

   ext_col  int_col  accident  clean_title  
0    262.0     10.0       0.0          0.0  
1     38.0     71.0       1.0          1.0  
2     29.0     14.0       1.0          1.0  
3     29.0     10.0       1.0          1.0  
4    303.0    155.0       1.0          1.0  
X test
   brand   model  model_year  milage  fuel_type  engine  transmission  \
0   26.0  1390.0        2015   98000        3.0   325.0          16.0   
1   26.0  1377.0        2020    9142        4.0   787.0          31.0   
2   14.0   636.0        2022   28121        3.0   540.0  

We decided to remove the outliers, basically the top and bottom standard deviations of mileage and price as this would remove the million dollar luxury cars and the extremely cheap junkers in hope to make our model better

In [223]:
# Splitting the data
# for now we just focused on train since it is so large over 175 thousand entries
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Standardizing the features using StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [237]:
my_linreg = LinearRegression()

my_linreg.fit(X_train_scaled, y_train)

# Predict on scaled X_test
y_predict_lr = my_linreg.predict(X_test_scaled)

print("True values vs Predicted values:")
for true_val, pred_val in zip(y_test[:25], y_predict_lr[:25]):
    print(f" {true_val} | {pred_val}")

True values vs Predicted values:
 49000 | 31555.001597155886
 6499 | 13858.058190363223
 45900 | 38751.80368142426
 34000 | 56958.694492254115
 13000 | 11437.344464287424
 73000 | 59180.477412932916
 8500 | 22843.987666641857
 69995 | 44734.1495643864
 41000 | 46519.46257856163
 45500 | 44006.15236976418
 46500 | 57739.87096832947
 24900 | 50115.238223128836
 20500 | 24753.288345093206
 65990 | 61507.285485524975
 28900 | 38607.3364846346
 9000 | 15696.297192249323
 2500 | 10035.256550124832
 39500 | 46978.88546537433
 28000 | 39791.222348999894
 22500 | 28374.758095564714
 29000 | 33599.32355495178
 24000 | 43185.46733049717
 34000 | 41404.83782676458
 30000 | 50682.358051814066
 68590 | 46461.98025324329


In [239]:
# Evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_predict_lr)
rmse = np.sqrt(mse)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)

Mean Squared Error: 205557598.60933372
Root Mean Squared Error: 14337.280028280598
